### 中间件
    中间件是 Django 请求/响应处理的钩子框架。它是一个轻量级的、低级的“插件”系统，用于全局改变 Django 的输入或输出。
    中间件定义了request请求到达视图函数之前，以及视图函数执行之后，response响应返回客户端之前执行的额外处理。

### 自定义中间件
    中间件可以被定义在项目的任何路径下，一般可在应用下新建一个文件夹存储自定义的中间件。如myapp/my_middleware.
    目前主要有函数实现和类实现两种方式定义中间件
    
#### 方式一.函数实现
    该方式实现的中间件是一个嵌套函数，外部函数名称即为中间件名称，其接收get_response作为参数。
    内部嵌套一个接收request作为参数的函数，名称可自行指定。
    外部函数定义的代码在django服务器启动时便会立刻执行，且只执行一次。
    内部函数定义的代码在服务器接收到请求时均会执行。
    因此，初始化代码应放在外部函数，相当于类的__init__方法；而针对不同客户端请求的代码应放在内部函数。
    在内部方法中，get_response()方法会调用request对应的视图函数。因此以get_response()方法为界，在此方法前的代码会在视图函数执行前完成，此方法后的代码会在视图函数执行后再被执行。

In [ ]:
# 官方文档示例
def simple_middleware(get_response):
    # One-time configuration and initialization.
    # 此处定义初始化代码

    def middleware(request):
        # Code to be executed for each request before
        # the view (and later middleware) are called.
        #视图函数执行前调用的代码，每个request请求都会使此处的代码被执行

        response = get_response(request)

        # Code to be executed for each request/response after
        # the view is called.
        #视图函数执行后调用的代码，每个request请求都会使此处的代码被执行

        return response

    return middleware

#### 方式二.类实现
    __init__方法用于初始化代码
    __call__方法定义核心代码，同样以get_response()为界，上方为执行视图函数前执行的代码，下方为执行视图函数后执行的代码。

In [ ]:
# 官方文档代码示例
class SimpleMiddleware:
    def __init__(self, get_response):
        self.get_response = get_response
        # One-time configuration and initialization.

    def __call__(self, request):
        # Code to be executed for each request before
        # the view (and later middleware) are called.

        response = self.get_response(request)

        # Code to be executed for each request/response after
        # the view is called.

        return response

#### 即将被遗弃的中间件定义方式
    原始的定义中间件的方式为定义一个类，继承MiddleWareMixin,将函数执行前后的代码分别放在两个函数中。
    视图执行前被调用的方法：
        process_request(self, request)
    视图函数执行后被调用的方法：
        process_response(self, request, response)

### 激活中间件
    若要使用自定义的中间件，则应在settings.py中MIDDLEWARE列表中登记激活。

In [ ]:
# settings.py
MIDDLEWARE = [
    'django.middleware.security.SecurityMiddleware',
    'django.contrib.sessions.middleware.SessionMiddleware',
    'django.middleware.common.CommonMiddleware',
    'django.middleware.csrf.CsrfViewMiddleware',
    'django.contrib.auth.middleware.AuthenticationMiddleware',
    'django.contrib.messages.middleware.MessageMiddleware',
    'django.middleware.clickjacking.XFrameOptionsMiddleware',
    # 激活中间件
    'myapp.my_middleware.SimpleMiddleware',
    
]

### 中间件的顺序与分层
    引用官方文档的说明：
    在请求阶段，在调用视图之前，Django 按照定义的顺序应用中间件 MIDDLEWARE，自顶向下。
    可以把它想象成一个洋葱：每个中间件类都是一个“层”，它覆盖了洋葱的核心。如果请求通过洋葱的所有层（每一个调用 get_response ）以将请求传递到下一层，一直到内核的视图，那么响应将在返回的过程中通过每个层（以相反的顺序）。
    如果其中一层决定停止并返回响应而不调用get_response，那么该层(包括视图)中的洋葱层都不会看到请求或响应。响应将只通过请求传入的相同层返回。